# 🔍 Spotify Crash Analysis

This notebook analyzes exported Spotify data to identify patterns in crashes and playback errors.

**What it analyzes:**
- Playback errors (fatal and non-fatal)
- Audio driver errors
- Track stuck events
- Connection errors
- Patterns by date, app version, tracks, etc.


## 1️⃣ Setup


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Style
plt.style.use('default')
sns.set_palette("husl")

DATA_DIR = Path("..") / "data"
TECHNICAL_LOG_DIR = DATA_DIR / "Spotify Technical Log Information"

print(f"📁 Loading data from: {DATA_DIR.resolve()}")


📁 Loading data from: /Users/aryamaan/Desktop/Projects/spotim8/data


## 2️⃣ Load Error Data


In [2]:
# Load playback errors from parquet (if exists)
playback_errors_path = DATA_DIR / "playback_errors.parquet"
if playback_errors_path.exists():
    errors_df = pd.read_parquet(playback_errors_path)
    print(f"✅ Loaded {len(errors_df)} playback errors from parquet")
else:
    # Load directly from JSON
    error_file = TECHNICAL_LOG_DIR / "PlaybackError.json"
    if error_file.exists():
        with open(error_file, 'r', encoding='utf-8') as fp:
            errors_data = json.load(fp)
        errors_df = pd.DataFrame(errors_data)
        if 'timestamp_utc' in errors_df.columns:
            errors_df['timestamp'] = pd.to_datetime(errors_df['timestamp_utc'], utc=True, errors='coerce')
        print(f"✅ Loaded {len(errors_df)} playback errors from JSON")
    else:
        errors_df = pd.DataFrame()
        print("⚠️  No playback errors found")

if len(errors_df) > 0:
    print(f"\n📊 Error Summary:")
    print(f"   Total errors: {len(errors_df)}")
    if 'message_fatal' in errors_df.columns:
        fatal_count = errors_df['message_fatal'].sum()
        print(f"   Fatal errors: {fatal_count}")
        print(f"   Non-fatal errors: {len(errors_df) - fatal_count}")
    if 'message_error_code' in errors_df.columns:
        print(f"\n   Error codes:")
        print(errors_df['message_error_code'].value_counts().to_string())


✅ Loaded 9 playback errors from parquet

📊 Error Summary:
   Total errors: 9
   Fatal errors: 2
   Non-fatal errors: 7

   Error codes:
message_error_code
8001    7
12      2


## 3️⃣ Analyze Fatal Errors (Crashes)


In [3]:
if len(errors_df) > 0 and 'message_fatal' in errors_df.columns:
    fatal_errors = errors_df[errors_df['message_fatal'] == True].copy()
    
    if len(fatal_errors) > 0:
        print("=" * 60)
        print(f"🚨 FATAL ERRORS (Crashes): {len(fatal_errors)}")
        print("=" * 60)
        
        for idx, row in fatal_errors.iterrows():
            print(f"\n📅 Date: {row.get('timestamp_utc', row.get('timestamp', 'N/A'))}")
            print(f"   Error Code: {row.get('message_error_code', 'N/A')}")
            print(f"   Track: {row.get('message_track_id', 'N/A')}")
            print(f"   Position: {row.get('message_position_ms', 0) / 1000:.1f}s")
            print(f"   App Version: {row.get('context_application_version', 'N/A')}")
            print(f"   OS Version: {row.get('context_os_version', 'N/A')}")
            print(f"   Bitrate: {row.get('message_bitrate', 'N/A')}")
            print(f"   Codec: {row.get('message_audiocodec', 'N/A')}")
    else:
        print("✅ No fatal errors found")
else:
    print("⚠️  No error data available")


🚨 FATAL ERRORS (Crashes): 2

📅 Date: 2025-12-02T06:09:49.124Z
   Error Code: 12
   Track: spotify:track:58k32my5lKofeZRtIvBDg9
   Position: 141.9s
   App Version: 9.1.0.1151
   OS Version: 18.6.2
   Bitrate: 96000
   Codec: vorbis

📅 Date: 2025-11-27T04:39:04.581Z
   Error Code: 12
   Track: spotify:track:086THPnabbu1zfDjRsxpoN
   Position: 62.3s
   App Version: 9.0.98.1187
   OS Version: 18.6.2
   Bitrate: 96000
   Codec: vorbis


## 4️⃣ Load Audio Driver Errors


In [4]:
# Load audio driver errors
audio_error_file = TECHNICAL_LOG_DIR / "AudioDriverError.json"
if audio_error_file.exists():
    with open(audio_error_file, 'r', encoding='utf-8') as fp:
        audio_errors_data = json.load(fp)
    audio_errors_df = pd.DataFrame(audio_errors_data)
    if 'timestamp_utc' in audio_errors_df.columns:
        audio_errors_df['timestamp'] = pd.to_datetime(audio_errors_df['timestamp_utc'], utc=True, errors='coerce')
    print(f"✅ Loaded {len(audio_errors_df)} audio driver errors")
    
    print(f"\n📊 Audio Driver Error Summary:")
    if 'message_error_code' in audio_errors_df.columns:
        print(f"\n   Error codes:")
        print(audio_errors_df['message_error_code'].value_counts().to_string())
    if 'message_location' in audio_errors_df.columns:
        print(f"\n   Error locations:")
        print(audio_errors_df['message_location'].value_counts().head(10).to_string())
    if 'message_driver_name' in audio_errors_df.columns:
        print(f"\n   Drivers:")
        print(audio_errors_df['message_driver_name'].value_counts().to_string())
else:
    audio_errors_df = pd.DataFrame()
    print("⚠️  No audio driver errors found")


✅ Loaded 6 audio driver errors

📊 Audio Driver Error Summary:

   Error codes:
message_error_code
-50    3
 0     3

   Error locations:
message_location
AudioUnitDriver2:850 AVAudioSessionMediaServicesWereResetNotification    3
pausePlayback                                                            1
AudioUnitDriver2:800 AudioOutputUnitStart(_outputUnit)                   1
AudioUnitDriver2:689 AudioUnitInitialize(_outputUnit)                    1

   Drivers:
message_driver_name
AudioUnitDriver2    6


## 5️⃣ Correlation Analysis - Fatal Errors & Audio Driver Issues


In [5]:
print("📊 Correlation Analysis:")
print("\n" + "=" * 60)

# Check if fatal errors correlate with specific tracks
if len(errors_df) > 0 and 'message_fatal' in errors_df.columns:
    fatal_errors = errors_df[errors_df['message_fatal'] == True]
    if len(fatal_errors) > 0:
        print("\n🚨 FATAL ERROR ANALYSIS:")
        print(f"   Total fatal errors: {len(fatal_errors)}")
        
        if 'message_error_code' in fatal_errors.columns:
            print(f"\n   Fatal error codes:")
            print(f"   {fatal_errors['message_error_code'].value_counts().to_string()}")
        
        if 'message_track_id' in fatal_errors.columns:
            print(f"\n   Tracks with fatal errors:")
            fatal_tracks = fatal_errors['message_track_id'].value_counts()
            for track_id, count in fatal_tracks.items():
                print(f"      {count}x - {track_id}")
        
        if 'context_application_version' in fatal_errors.columns:
            print(f"\n   App versions with fatal errors:")
            fatal_versions = fatal_errors['context_application_version'].value_counts()
            for version, count in fatal_versions.items():
                print(f"      {count}x - {version}")

# Check audio driver errors around fatal errors
if len(audio_errors_df) > 0 and len(errors_df) > 0:
    if 'timestamp' in audio_errors_df.columns and 'timestamp' in errors_df.columns:
        fatal_errors = errors_df[errors_df.get('message_fatal', False) == True]
        if len(fatal_errors) > 0:
            print(f"\n🔍 Audio Driver Errors Around Fatal Errors:")
            
            for idx, fatal_row in fatal_errors.iterrows():
                fatal_time = fatal_row.get('timestamp')
                if pd.notna(fatal_time):
                    # Find audio errors within 5 seconds
                    time_window = pd.Timedelta(seconds=5)
                    nearby_audio = audio_errors_df[
                        (audio_errors_df['timestamp'] >= fatal_time - time_window) &
                        (audio_errors_df['timestamp'] <= fatal_time + time_window)
                    ]
                    
                    if len(nearby_audio) > 0:
                        print(f"\n   Fatal error at {fatal_time}:")
                        print(f"      Found {len(nearby_audio)} nearby audio driver errors")
                        for audio_idx, audio_row in nearby_audio.iterrows():
                            print(f"         Error code: {audio_row.get('message_error_code', 'N/A')}")
                            print(f"         Location: {audio_row.get('message_location', 'N/A')}")

print("\n" + "=" * 60)


📊 Correlation Analysis:


🚨 FATAL ERROR ANALYSIS:
   Total fatal errors: 2

   Fatal error codes:
   message_error_code
12    2

   Tracks with fatal errors:
      1x - spotify:track:58k32my5lKofeZRtIvBDg9
      1x - spotify:track:086THPnabbu1zfDjRsxpoN

   App versions with fatal errors:
      1x - 9.1.0.1151
      1x - 9.0.98.1187

🔍 Audio Driver Errors Around Fatal Errors:

   Fatal error at 2025-12-02 06:09:49.124000+00:00:
      Found 4 nearby audio driver errors
         Error code: -50
         Location: pausePlayback
         Error code: -50
         Location: AudioUnitDriver2:800 AudioOutputUnitStart(_outputUnit)
         Error code: -50
         Location: AudioUnitDriver2:689 AudioUnitInitialize(_outputUnit)
         Error code: 0
         Location: AudioUnitDriver2:850 AVAudioSessionMediaServicesWereResetNotification

   Fatal error at 2025-11-27 04:39:04.581000+00:00:
      Found 2 nearby audio driver errors
         Error code: 0
         Location: AudioUnitDriver2:850 AVA

## 6️⃣ Recommendations


In [6]:
print("💡 RECOMMENDATIONS:")
print("\n" + "=" * 60)

recommendations = []

# Check for error code 12 (fatal)
if len(errors_df) > 0 and 'message_error_code' in errors_df.columns:
    error_12 = errors_df[errors_df['message_error_code'] == 12]
    if len(error_12) > 0:
        recommendations.append("\n🚨 Error Code 12 (Fatal):")
        recommendations.append("   This is a fatal playback error. Common causes:")
        recommendations.append("   - Corrupted audio file on Spotify's servers")
        recommendations.append("   - Network connectivity issues")
        recommendations.append("   - Audio driver/system issues")
        recommendations.append(f"   - Found {len(error_12)} occurrences")

# Check for error code 8001
if len(errors_df) > 0 and 'message_error_code' in errors_df.columns:
    error_8001 = errors_df[errors_df['message_error_code'] == 8001]
    if len(error_8001) > 0:
        recommendations.append("\n⚠️  Error Code 8001 (Non-fatal):")
        recommendations.append("   This is a non-fatal playback error. Common causes:")
        recommendations.append("   - Temporary network issues")
        recommendations.append("   - Streaming buffer problems")
        recommendations.append(f"   - Found {len(error_8001)} occurrences")

# Check for audio driver error -50
if len(audio_errors_df) > 0 and 'message_error_code' in audio_errors_df.columns:
    error_neg50 = audio_errors_df[audio_errors_df['message_error_code'] == -50]
    if len(error_neg50) > 0:
        recommendations.append("\n🔧 Audio Driver Error -50:")
        recommendations.append("   This is a macOS/iOS system error (kAudioUnitErr_InvalidProperty).")
        recommendations.append("   Common causes:")
        recommendations.append("   - Audio session conflicts")
        recommendations.append("   - System audio driver issues")
        recommendations.append("   - App trying to access audio unit while it's being reset")
        recommendations.append(f"   - Found {len(error_neg50)} occurrences")

# Check for AVAudioSessionMediaServicesWereResetNotification
if len(audio_errors_df) > 0 and 'message_location' in audio_errors_df.columns:
    reset_notifications = audio_errors_df[audio_errors_df['message_location'].str.contains('AVAudioSessionMediaServicesWereResetNotification', na=False)]
    if len(reset_notifications) > 0:
        recommendations.append("\n🔄 Audio Session Reset Notifications:")
        recommendations.append("   iOS is resetting the audio session. This can cause crashes.")
        recommendations.append("   Common causes:")
        recommendations.append("   - Another app taking control of audio")
        recommendations.append("   - System audio interruptions (calls, alarms)")
        recommendations.append("   - Audio hardware changes")
        recommendations.append(f"   - Found {len(reset_notifications)} occurrences")

# Check for problematic tracks
if len(errors_df) > 0 and 'message_track_id' in errors_df.columns:
    track_error_counts = errors_df['message_track_id'].value_counts()
    problematic_tracks = track_error_counts[track_error_counts >= 2]
    if len(problematic_tracks) > 0:
        recommendations.append("\n🎵 Problematic Tracks:")
        recommendations.append("   These tracks have multiple errors:")
        for track_id, count in problematic_tracks.head(5).items():
            recommendations.append(f"   - {track_id}: {count} errors")
        recommendations.append("   Consider removing these from playlists or reporting to Spotify.")

# App version issues
if len(errors_df) > 0 and 'context_application_version' in errors_df.columns:
    version_errors = errors_df.groupby('context_application_version').size()
    if len(version_errors) > 1:
        worst_version = version_errors.idxmax()
        worst_count = version_errors.max()
        recommendations.append(f"\n📱 App Version Issues:")
        recommendations.append(f"   Version {worst_version} has the most errors ({worst_count}).")
        recommendations.append("   Consider updating to the latest version.")

if recommendations:
    for rec in recommendations:
        print(rec)
else:
    print("\n✅ No specific issues identified. Your Spotify app appears to be working normally.")

print("\n" + "=" * 60)
print("\n💡 General Tips:")
print("   1. Keep your Spotify app updated to the latest version")
print("   2. Restart your phone if crashes persist")
print("   3. Check your internet connection quality")
print("   4. Clear Spotify app cache (Settings > Apps > Spotify > Clear Cache)")
print("   5. Reinstall the app if crashes are frequent")
print("   6. Report problematic tracks to Spotify support")


💡 RECOMMENDATIONS:


🚨 Error Code 12 (Fatal):
   This is a fatal playback error. Common causes:
   - Corrupted audio file on Spotify's servers
   - Network connectivity issues
   - Audio driver/system issues
   - Found 2 occurrences

⚠️  Error Code 8001 (Non-fatal):
   This is a non-fatal playback error. Common causes:
   - Temporary network issues
   - Streaming buffer problems
   - Found 7 occurrences

🔧 Audio Driver Error -50:
   This is a macOS/iOS system error (kAudioUnitErr_InvalidProperty).
   Common causes:
   - Audio session conflicts
   - System audio driver issues
   - App trying to access audio unit while it's being reset
   - Found 3 occurrences

🔄 Audio Session Reset Notifications:
   iOS is resetting the audio session. This can cause crashes.
   Common causes:
   - Another app taking control of audio
   - System audio interruptions (calls, alarms)
   - Audio hardware changes
   - Found 3 occurrences

🎵 Problematic Tracks:
   These tracks have multiple errors:
   - spotif